### Imports

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


### Global Variables

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
engine = create_engine('sqlite:///./files/data.sqlite')

### ETL Process

##### 1. Extracting necessary data from table into a dataframe

In [9]:
page = BeautifulSoup(requests.get(url=url).text, 'lxml')
table = page.find('table', attrs={'class':'wikitable'})
df = pd.read_html(str(table))[0]
df

Country/Territory UN region IMF[1][13]            World Bank[14]  \
    Country/Territory UN region   Forecast       Year       Estimate   
0               World         —  104476432       2023      100562011   
1       United States  Americas   26949643       2023       25462700   
2               China      Asia   17700899  [n 1]2023       17963171   
3             Germany    Europe    4429838       2023        4072192   
4               Japan      Asia    4230862       2023        4231141   
..                ...       ...        ...        ...            ...   
209             Palau   Oceania        267       2023              —   
210          Kiribati   Oceania        246       2023            223   
211             Nauru   Oceania        150       2023            151   
212        Montserrat  Americas          —          —              —   
213            Tuvalu   Oceania         63       2023             60   

               United Nations[15]             
          Year           Estimate       Year  
0         2022           96698005       2021  
1         2022           23315081       2021  
2    [n 3]2022           17734131  [n 1]2021  
3         2022            4259935       2021  
4         2022            4940878       2021  
..         ...                ...        ...  
209          —                218       2021  
210       2022                227       2021  
211       2022                155       2021  
212          —                 72       2021  
213       2022                 60       2021  

[214 rows x 8 columns]

##### 2. Treating data removing unnecessary columns and values, changing columns and values

In [10]:
df = df.drop(columns=[('UN region', 'UN region'),
    ('IMF[1][13]', 'Year'), ('World Bank[14]', 'Estimate'),
    ('World Bank[14]', 'Year'), ('United Nations[15]', 'Estimate'),
    ('United Nations[15]', 'Year')])
df.columns = [col for col in df.columns.values]
columns = df.columns.tolist()
columns[0] = 'Country'
columns[1] = 'GDP_USD_millions'
df.columns = columns
df = df[(df['GDP_USD_millions'] != '—') & (df['Country'] != 'World')]
df = df.reset_index(drop=True)
df['GDP_USD_millions'] = df['GDP_USD_millions'].astype(float)
df['GDP_USD_millions'] = np.round(df['GDP_USD_millions'] / 1000,2)
df = df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})
df

,Country,GDP_USD_billions
0,United States,26949.64
1,China,17700.90
2,Germany,4429.84
3,Japan,4230.86
4,India,3732.22
...,...,...
186,Marshall Islands,0.28
187,Palau,0.27
188,Kiribati,0.25
189,Nauru,0.15


##### 3. Exporting the data into a .csv and a .sqlite

In [13]:
df.to_csv(path_or_buf=r'./files/data.csv')

df.to_sql(name='GDP', con=engine, if_exists='replace', index=False)